In [1]:
import json 
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

## Loading openai key
with open('../environ.json') as f:
    config = json.load(f)
    api_keys = list(config.values())[0]

# assign openai key to an environment variable
os.environ["OPENAI_API_KEY"] = api_keys

# # check to confirm
# os.getenv("OPENAI_API_KEY")


In [3]:
# establish connection to neo4j graph database
graph = Neo4jGraph(
    url="bolt://localhost:7689", 
    username="admin", 
    password="cran2graph", 
    database="neo4j"
)
print(graph.schema)

Node properties are the following:
Person {name: STRING},Package {latestPublishedDate: STRING, name: STRING, md5sum: STRING, description: STRING, license: STRING, version: STRING},Organization {name: STRING},License {type: STRING},Institution {type: STRING}
Relationship properties are the following:

The relationships are the following:
(:Person)-[:WORKS_AT]->(:Institution),(:Person)-[:CONTRIBUTED_TO]->(:Package),(:Person)-[:MAINTAINS]->(:Package),(:Package)-[:DEPENDS_ON]->(:Package),(:Package)-[:LICENSE_BY]->(:License),(:Package)-[:SUPPORTED_BY]->(:Organization)


In [4]:
# check the graph schema
print(graph.schema)

Node properties are the following:
Person {name: STRING},Package {latestPublishedDate: STRING, name: STRING, md5sum: STRING, description: STRING, license: STRING, version: STRING},Organization {name: STRING},License {type: STRING},Institution {type: STRING}
Relationship properties are the following:

The relationships are the following:
(:Person)-[:WORKS_AT]->(:Institution),(:Person)-[:CONTRIBUTED_TO]->(:Package),(:Person)-[:MAINTAINS]->(:Package),(:Package)-[:DEPENDS_ON]->(:Package),(:Package)-[:LICENSE_BY]->(:License),(:Package)-[:SUPPORTED_BY]->(:Organization)


In [5]:
# chain rule
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

c:\Users\Dee\root\Projects\dev\devGraph\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
chain.run("list all packages Kirill Müller contributed to")

c:\Users\Dee\root\Projects\dev\devGraph\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Kirill Müller'})-[:CONTRIBUTED_TO]->(pkg:Package)
RETURN pkg;
Full Context:
[{'pkg': {'license': 'GPL-2', 'latestPublishedDate': '2021-12-06', 'md5sum': 'b373f83a971b68402825e699fce7cb4d', 'name': 'sessioninfo', 'description': 'Query and print information about the current R\n    session.  It is similar to utils::sessionInfo(), but includes more\n    information about packages, and where they were installed from.', 'version': '1.2.2'}}, {'pkg': {'license': 'MIT + file LICENSE', 'latestPublishedDate': '2023-03-20', 'md5sum': 'e4ccfbe28c84b11f86388379c8b43925', 'name': 'tibble', 'description': 'Provides a tbl_df class (the tibble) with stricter checking and better formatting than the traditional\n    data frame.', 'version': '3.2.1'}}, {'pkg': {'license': 'MIT + file LICENSE', 'latestPublishedDate': '2018-03-13', 'md5sum': 'cfa02c563196a79bf8bb4db2e66585fd', 'name': 'bindr', 'description': 'Provides a 

"I don't know the answer."

In [7]:
chain.run("How many packages is maintained by Kirill Müller?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:MAINTAINS]->(p:Package)
RETURN COUNT(p)
Full Context:
[{'COUNT(p)': 28}]

> Finished chain.


'Kirill Müller maintains 28 packages.'

In [9]:
chain.run("how many packages does Jing Zhang maintains and the names and when it was published?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Jing Zhang'})-[:MAINTAINS]->(pkg:Package)
RETURN COUNT(pkg) AS packageCount, pkg.name AS packageName, pkg.latestPublishedDate AS publishedDate
Full Context:
[{'packageCount': 1, 'packageName': 'ggrisk', 'publishedDate': '2021-08-09'}, {'packageCount': 1, 'packageName': 'do', 'publishedDate': '2021-08-03'}, {'packageCount': 1, 'packageName': 'msig', 'publishedDate': '2021-06-29'}, {'packageCount': 1, 'packageName': 'base.rms', 'publishedDate': '2020-08-01'}, {'packageCount': 1, 'packageName': 'cutoff', 'publishedDate': '2019-12-20'}, {'packageCount': 1, 'packageName': 'nomogramFormula', 'publishedDate': '2020-01-28'}]

> Finished chain.


"Jing Zhang maintains a total of 6 packages. The names of the packages are 'ggrisk', 'do', 'msig', 'base.rms', 'cutoff', and 'nomogramFormula'. Here are the published dates for each package: 'ggrisk' was published on August 9, 2021, 'do' was published on August 3, 2021, 'msig' was published on June 29, 2021, 'base.rms' was published on August 1, 2020, 'cutoff' was published on December 20, 2019, and 'nomogramFormula' was published on January 28, 2020."

In [11]:
chain.run("Name of package Dennis Irorere contributed to")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Dennis Irorere'})-[:CONTRIBUTED_TO]->(p:Package)
RETURN p.name
Full Context:
[{'p.name': 'anyflights'}]

> Finished chain.


'Dennis Irorere contributed to the package named "anyflights."'

In [14]:
chain.run("how many packages does Kirill Müller contributes and list the packages. I might have spelt name wrong")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Kirill Müller'})-[:CONTRIBUTED_TO]->(pkg:Package)
RETURN COUNT(pkg) AS numberOfPackages, COLLECT(pkg.name) AS packageList
Full Context:
[{'numberOfPackages': 54, 'packageList': ['RSQLite', 'here', 'RMariaDB', 'DBI', 'chk', 'roxygen2md', 'mlfit', 'StrucDiv', 'term', 'dm', 'DBItest', 'log4r', 'hms', 'skimr', 'bindrcpp', 'desc', 'indiedown', 'enc', 'knitrProgressBar', 'fledge', 'cli', 'tibble', 'universals', 'tibblify', 'styler', 'sessioninfo', 'tikzDevice', 'pillar', 'duckplyr', 'winch', 'withr', 'memoise', 'wrswoR', 'lest', 'drake', 'MakefileR', 'optparse', 'procmaps', 'traudem', 'utf8', 'dplyr', 'kimisc', 'extras', 'plogr', 'covr', 'profile', 'bindr', 'RPostgres', 'whereami', 'blob', 'sf', 'mockr', 'igraph', 'rprojroot']}]

> Finished chain.


'Kirill Müller contributes to a total of 54 packages. Some of the packages he contributes to include RSQLite, here, RMariaDB, DBI, chk, roxygen2md, mlfit, StrucDiv, term, dm, DBItest, log4r, hms, skimr, bindrcpp, desc, indiedown, enc, knitrProgressBar, fledge, cli, tibble, universals, tibblify, styler, sessioninfo, tikzDevice, pillar, duckplyr, winch, withr, memoise, wrswoR, lest, drake, MakefileR, optparse, procmaps, traudem, utf8, dplyr, kimisc, extras, plogr, covr, profile, bindr, RPostgres, whereami, blob, sf, mockr, igraph, and rprojroot.'

In [10]:
chain.run("what package will you recommend for osm data processing using the description ")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Package)
WHERE p.description CONTAINS 'osm data processing'
RETURN p.name
Full Context:
[]

> Finished chain.


"I'm sorry, but without any provided information, I don't have a specific package to recommend for OSM data processing. However, there are several popular packages available for this purpose, such as Osmosis, osmium-tool, and GDAL. These packages offer various functionalities for working with OSM data, including data extraction, filtering, and conversion. It would be helpful to consider your specific requirements and the programming language you are using to determine the most suitable package for your needs."

In [13]:
chain.run("what dependency has the most package depending on it and the maintainer")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Package)-[:DEPENDS_ON]->(d:Package)
WITH d, COUNT(p) AS dependencyCount
ORDER BY dependencyCount DESC
LIMIT 1
MATCH (d)<-[:MAINTAINS]-(m:Person)
RETURN d.name AS dependency, m.name AS maintainer
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have access to the information about the dependency with the most packages depending on it and its maintainer."

In [46]:
chain.run("using description properties recommend an alternative package for dplyr")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Package)-[:DEPENDS_ON]->(d:Dependency)<-[:DEPENDS_ON]-(p2:Package)
WHERE p1.package = 'dplyr'
RETURN p2.package, p2.description
Full Context:
[{'p2.package': 'dials', 'p2.description': 'Many models contain tuning parameters (i.e. parameters that\n    cannot be directly estimated from the data). These tools can be used\n    to define objects for creating, simulating, or validating values for\n    such parameters.'}, {'p2.package': 'googledrive', 'p2.description': 'Manage Google Drive files from R.'}, {'p2.package': 'retroharmonize', 'p2.description': 'Assist in reproducible retrospective (ex-post) harmonization\n    of data, particularly individual level survey data, by providing tools\n    for organizing metadata, standardizing the coding of variables, and\n    variable names and value labels, including missing values, and\n    documenting the data transformations, with the help of comprehensive\n    s3 classes.'}

'Based on the description properties, an alternative package for dplyr could be "dbplyr". It is a dplyr back end for databases that allows you to work with remote database tables as if they are in-memory data frames. While basic features work with any database that has a DBI back end, more advanced features require SQL translation to be provided by the package author.'

In [24]:
def evaluate_accuracy(chain, questions, expected_answers):
    num_correct = 0
    total_questions = len(questions)
    
    for i, question in enumerate(questions):
        generated_answer = chain.run(question)
        expected_answer = expected_answers[i]
        
        if expected_answer in generated_answer:
            num_correct += 1
            
    accuracy = num_correct / total_questions
    return accuracy

questions = [
    "How many packages is maintained by Kirill Müller?",
    "How many packages has Kirill Müller contributed to?",
    "Name of package Dennis Irorere contributed to",
    "Who maintains anyflights package",
    "Who is maintaining the most packages?",
    "Who is contributing to the most to packages?",
    'What package have the most contributions',
    "What packages depend on the most?"
]

expected_answers = [
    "28",
    "54",
    "anyflights",
    "Simon P. Couch",
    "Dirk Eddelbuettel",
    "Hadley Wickham",
    "mlpack",
    "stats"
]

In [25]:
accuracy = evaluate_accuracy(chain, questions , expected_answers)
print("Accuracy:", accuracy)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:MAINTAINS]->(p:Package)
RETURN COUNT(p) as numberOfPackagesMaintained;
Full Context:
[{'numberOfPackagesMaintained': 28}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: 'Kirill Müller'})-[:CONTRIBUTED_TO]->(p:Package)
RETURN COUNT(p) as num_packages_contributed
Full Context:
[{'num_packages_contributed': 54}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: "Dennis Irorere"})-[:CONTRIBUTED_TO]->(p:Package)
RETURN p.name
Full Context:
[{'p.name': 'anyflights'}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Package {name: "anyflights"})<-[:MAINTAINS]-(person:Person)
RETURN person;
Full Context:
[{'person': {'name': 'Simon P. Couch'}}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person

In [ ]:
num_correct

In [2]:
questions = [
    "How many packages is maintained by Kirill Müller?",
    "How many packages has Kirill Müller contributed to?",
    "Name of package Dennis Irorere contributed to",
    "Who maintains anyflights package",
    "Who is maintaining the most packages?",
    "Who is contributing to the most to packages?",
    "What are the packages that are supported by the most?",
    "What packages depend on the most?"
]


In [21]:
chain.run('What package is depend on the most?')



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Package)-[:DEPENDS_ON]->(p:Package)
RETURN p.name, COUNT(*) AS dependency_count
ORDER BY dependency_count DESC
LIMIT 1;
Full Context:
[{'p.name': 'stats', 'dependency_count': 5488}]

> Finished chain.


"The package that is depend on the most is 'stats' with a dependency count of 5488."

In [13]:
chain.run('Name of package Dennis Irorere contributed to')



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Dennis Irorere"})-[:CONTRIBUTED_TO]->(pkg:Package)
RETURN pkg.name
Full Context:
[{'pkg.name': 'anyflights'}]

> Finished chain.


'Dennis Irorere contributed to the package named anyflights.'

In [15]:
a = chain.run('Who maintains anyflights package')



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:MAINTAINS]->(pkg:Package {name: "anyflights"})
RETURN p
Full Context:
[{'p': {'name': 'Simon P. Couch'}}]

> Finished chain.


In [16]:
a

'Simon P. Couch maintains the anyflights package.'

In [23]:
accuracy = evaluate_accuracy(questions , expected_answers)
print("Accuracy:", accuracy)

TypeError: evaluate_accuracy() takes 2 positional arguments but 3 were given